# Binance API

https://api.binance.com/api/v3/

Endpoint:
- **/klines**

Description:
Get historical candlestick data for a specific symbol.

Parameters:
- symbol: The trading pair symbol to get data for (e.g., BTCUSDT).
- interval: The time interval for each candlestick (e.g., 1m, 5m, 1h).
- startTime: The start time for the data (in Milliseconds (Unix Epoch)).
- endTime: The end time for the data (in Milliseconds (Unix Epoch)).
- limit: The maximum number of candlesticks to return (default: 500, max: 1000).


### BTCUSDT

In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone

In [3]:
def get_binance_klines(symbol: str, interval: str = "1m", limit: int = 1000):
    """
    Fetches 1-minute OHLCV candles from Binance for a full previous day (1440 points split in two requests)
    """
    base_url = "https://api.binance.com/api/v3/klines"
    
    # Define yesterday's date range in UTC
    now_utc = datetime.utcnow().replace(tzinfo=timezone.utc)
    yesterday_start = (now_utc - timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
    yesterday_end = yesterday_start + timedelta(days=1)

    start_time_ms = int(yesterday_start.timestamp() * 1000)
    end_time_ms = int(yesterday_end.timestamp() * 1000)
    
    # First request (0–999)
    params_1 = {
        "symbol": symbol,
        "interval": interval,
        "startTime": start_time_ms,
        "limit": 1000
    }
    response_1 = requests.get(base_url, params=params_1)
    response_1.raise_for_status()
    data_1 = response_1.json()

    # Second request (1000–1439)
    last_time_ms = int(data_1[-1][0]) + 60_000  # Add one minute
    params_2 = {
        "symbol": symbol,
        "interval": interval,
        "startTime": last_time_ms,
        "endTime": end_time_ms,
        "limit": 440
    }
    response_2 = requests.get(base_url, params=params_2)
    response_2.raise_for_status()
    data_2 = response_2.json()

    # Combine and format
    all_data = data_1 + data_2
    df = pd.DataFrame(all_data, columns=[
        "timestamp", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "num_trades",
        "taker_buy_base_vol", "taker_buy_quote_vol", "ignore"
    ])

    df["timestamp"] = pd.to_datetime(df["timestamp"], unit='ms')
    df.set_index("timestamp", inplace=True)
    df = df[["open", "high", "low", "close", "volume"]].astype(float)

    return df


In [4]:
# 🟢 Example usage
btc_df = get_binance_klines("BTCUSDT")
print(btc_df.head())
print(f"Total rows: {len(btc_df)}")

                          open       high        low      close    volume
timestamp                                                                
2025-06-22 00:00:00  102120.02  102398.34  102120.01  102279.12  65.39842
2025-06-22 00:01:00  102279.12  102409.09  102265.38  102321.25  55.95674
2025-06-22 00:02:00  102321.25  102345.30  102249.70  102262.90  42.40999
2025-06-22 00:03:00  102262.90  102584.00  102262.90  102519.86  48.99128
2025-06-22 00:04:00  102519.86  102588.71  102519.86  102544.47  41.88131
Total rows: 1440
